In [4]:
install.packages("h2o")
library(h2o)
library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘RCurl’



----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit https://docs.h2o.ai

----------------------------------------------------------------------



Attaching package: ‘h2o’


The following objects are masked from ‘package:lubridate’:

    day, hour, month, week, year


The following objects are masked from ‘package:stats’:

    cor, sd, var


The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




In [24]:
library(h2o)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train <- h2o.importFile("https://s3.amazonaws.com/h2o-public-test-data/smalldata/higgs/higgs_train_10k.csv")
test <- h2o.importFile("https://s3.amazonaws.com/h2o-public-test-data/smalldata/higgs/higgs_test_5k.csv")

# Identify predictors and response
y <- "response"
x <- setdiff(names(train), y)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         14 minutes 49 seconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.44.0.3 
    H2O cluster version age:    10 months and 18 days 
    H2O cluster name:           H2O_started_from_R_root_qad537 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   3.17 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:                  R version 4.4.2 (2024-10-31) 


Warning message in h2o.clusterInfo():
“
Your H2O cluster version is (10 months and 18 days) old. There may be a newer version available.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html”



  |======================================================================| 100%
  |======================================================================| 100%


In [25]:
# For binary classification, response should be a factor
train[, y] <- as.factor(train[, y])
test[, y] <- as.factor(test[, y])

In [28]:
head(train)

,response,x1,x2,x3,x4,x5,x6,x7,x8,x9,⋯,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0.8692932,-0.6350818,0.225690261,0.3274701,-0.6899932,0.7542022,-0.2485731,-1.0920639,0.000000,⋯,-0.01045457,-0.0457671694,3.101961,1.3537600,0.9795631,0.9780762,0.9200048,0.7216575,0.9887509,0.8766783
2,1,0.9075421,0.3291473,0.359411865,1.4979699,-0.3130095,1.0955306,-0.5575249,-1.5882298,2.173076,⋯,-1.13893008,-0.0008191102,0.000000,0.3022199,0.8330482,0.9856997,0.9780984,0.7797322,0.9923558,0.7983426
3,1,0.7988347,1.4706388,-1.635974765,0.4537732,0.4256292,1.1048746,1.2823223,1.3816643,0.000000,⋯,1.12884760,0.9004608393,0.000000,0.9097533,1.1083305,0.9856922,0.9513313,0.8032515,0.8659244,0.7801176
4,0,1.3443848,-0.8766260,0.935912728,1.9920501,0.8824544,1.7860659,-1.6467777,-0.9423825,0.000000,⋯,-0.67837882,-1.3603563309,0.000000,0.9466525,1.0287037,0.9986561,0.7282806,0.8692002,1.0267365,0.9579040
5,1,1.1050090,0.3213556,1.522401214,0.8828076,-1.2053493,0.6814661,-1.0704639,-0.9218706,0.000000,⋯,-0.37356552,0.1130406111,0.000000,0.7558565,1.3610570,0.9866097,0.8380846,1.1332952,0.8722449,0.8084865
6,0,1.5958393,-0.6078107,0.007074916,1.8184496,-0.1119060,0.8475499,-0.5664370,1.5812393,2.173076,⋯,-0.65422696,-1.2743449211,3.101961,0.8237606,0.9381914,0.9717582,0.7891763,0.4305533,0.9613569,0.9578179


In [29]:
# Train & Cross-validate a GBM
my_gbm <- h2o.gbm(x = x,
                  y = y,
                  training_frame = train,
                  distribution = "bernoulli",
                  ntrees = 10,
                  max_depth = 3,
                  min_rows = 2,
                  learn_rate = 0.2,
                  nfolds = nfolds,
                  keep_cross_validation_predictions = TRUE,
                  seed = 1)

  |======================================================================| 100%


In [30]:
# Train & Cross-validate a RF
my_rf <- h2o.randomForest(x = x,
                          y = y,
                          training_frame = train,
                          ntrees = 50,
                          nfolds = nfolds,
                          keep_cross_validation_predictions = TRUE,
                          seed = 1)

  |======================================================================| 100%


In [ ]:
# Train a stacked ensemble using the GBM and RF above
ensemble <- h2o.stackedEnsemble(x = x,
                                y = y,
                                training_frame = train,
                                base_models = list(my_gbm, my_rf))

# Eval ensemble performance on a test set
perf <- h2o.performance(ensemble, newdata = test)


In [33]:
# Compare to base learner performance on the test set
perf_gbm_test <- h2o.performance(my_gbm, newdata = test)
perf_rf_test <- h2o.performance(my_rf, newdata = test)

print(sprintf("GBM Test AUC:  %s", h2o.auc(perf_gbm_test)))
print(sprintf("RF Test AUC:  %s", h2o.auc(perf_rf_test)))
print(sprintf("Ensemble Test AUC:  %s", h2o.auc(perf)))

[1] "GBM Test AUC:  0.752268622979435"
[1] "RF Test AUC:  0.769201346573838"
[1] "Ensemble Test AUC:  0.973863636363636"
